# Reading the data 

---

Lately, the _.fits_ data was transformed into _.csv_ and saved in database/csv_files. We are going to import a light curve sample and work with it.

In [1]:
# FILE_PATH = '/content/drive/MyDrive/01 - Iniciação Científica/02 - Datasets/exoplanets_confirmed/resampled_files/RESAMPLED_EN2_STAR_CHR_0101086161_20070516T060226_20071005T074409.csv'
FILE_PATH = 'C:/Users/guisa/Google Drive/01 - Iniciação Científica/02 - Datasets/exoplanets_confirmed/resampled_files/RESAMPLED_EN2_STAR_CHR_0105833549_20080415T231048_20080907T224903.csv'

In [2]:
import pandas as pd

data_sample = pd.read_csv(FILE_PATH)
display(data_sample)

DATE      WHITEFLUX
0      2008-04-16 11:17:01.652367  425517.134412
1      2008-04-16 11:30:51.915174  426870.998701
2      2008-04-16 11:44:42.177981  426445.431775
3      2008-04-16 11:58:32.440788  426497.707393
4      2008-04-16 12:12:22.703595  426530.518909
...                           ...            ...
15045  2008-09-08 01:05:25.581928  426348.291249
15046  2008-09-08 01:19:15.844734  424517.308637
15047  2008-09-08 01:33:06.107541  423893.013298
15048  2008-09-08 01:46:56.370348  423597.101577
15049  2008-09-08 02:00:46.633155  423233.988598

[15050 rows x 2 columns]

In [3]:
import numpy as np

x = data_sample.DATE.to_numpy()
y = data_sample.WHITEFLUX.to_numpy()

# Ideal Lowpass Filters

---

Um filtro bi-dimensional passa-baixa que deixa passar todas as frequências em um círculo de raio $D_0$ a partir da origem e remove todas as frequências fora desse círculo é chamado de filtro passa-baixa ideal (ILPF) e é descrito como

$$ H(u) = \begin{cases}
   1, &\text{if } D(u) \le D_0 \\
   0, &\text{if } D(u) \ge D_0
\end{cases} 
$$

onde $D_0$ é uma constante positiva, e $D(u)$ é a distância entre um ponto $u$ até o centro do retângulo de frequência, ou seja, é definido por

$$ D(u) = (u-P/2) $$

sendo $P$ o tamanho do vetor original preenchido (padded).

O ponto de transição entre $H(u) = 1$ e $H(u) = 0$ é chamado de frequência de corte

In [4]:
from tools import *

Loading BokehJS ...

In [5]:
def ideal_filter(array, fourier_transform, cutoff_freq):
  n_time = len(array)
  D0 = cutoff_freq * n_time
  xc = n_time

  for i in range(len(fourier_transform)):
    if fourier_transform[i] > D0:
      fourier_transform[i] = 0

  y_filtered = np.real(np.fft.ifft(fourier_transform))
  y_filtered += (array.mean() - y_filtered.mean())

  return y_filtered

In [6]:
def ideal_filter_result(array, cutoff_freq, numExpansion):
  Filter = filters.FrequencyDomainFiltering()

  Filter.expand_borders(y, numExpansion)
  y_expanded = Filter.getExpandedBorders

  y_filtered = ideal_filter(y_expanded, np.fft.fft(y_expanded), cutoff_freq)

  Filter.remove_expanded_borders(y_filtered, numExpansion)
  y_filtered = Filter.getNoExpanded

  return y_filtered

# Choosing the best Ideal Filter parameters

---



In [7]:
cutoff_freqs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [12]:
import time
t_o = time.time()

count = 0
for cutoff_freq in cutoff_freqs:
    # Filtering the curve
    Filter = filters.FrequencyDomainFiltering()
    Filter.filter(array=y, filter_technique='ideal', numExpansion=70, cutoff_freq=cutoff_freq, order=None)
    y_filtered = Filter.getFiltered

    # Salving filtered data
    PATH_DIR = 'C:/Users/guisa/Google Drive/01 - Iniciação Científica/02 - Datasets/exoplanets_confirmed/filters/ideal/cutoffFreq0'+str(int(cutoff_freq*10))       
    filters.export_results_csv(PATH_DIR, 'ideal', cutoff_freq, order=None)

    count +=1 
        
t_f = time.time()
print("It takes:", round(t_f-t_o, 2), "seconds to save all")

Saving filtered data for ideal, order = None and cutoff frequency = 0.1
All files have been saved sucessefuly

Saving filtered data for ideal, order = None and cutoff frequency = 0.2
All files have been saved sucessefuly

Saving filtered data for ideal, order = None and cutoff frequency = 0.3
All files have been saved sucessefuly

Saving filtered data for ideal, order = None and cutoff frequency = 0.4
All files have been saved sucessefuly

Saving filtered data for ideal, order = None and cutoff frequency = 0.5
All files have been saved sucessefuly

Saving filtered data for ideal, order = None and cutoff frequency = 0.6
All files have been saved sucessefuly

Saving filtered data for ideal, order = None and cutoff frequency = 0.7
All files have been saved sucessefuly

Saving filtered data for ideal, order = None and cutoff frequency = 0.8
All files have been saved sucessefuly

Saving filtered data for ideal, order = None and cutoff frequency = 0.9
All files have been saved sucessefuly

I

## Plotting some filters

In [9]:
cutoff_freq = 0.1

Filter = filters.FrequencyDomainFiltering()
Filter.filter(array=y, filter_technique='ideal', numExpansion=70, cutoff_freq=cutoff_freq, order=None)
y_filtered = Filter.getFiltered
title = f"Bessel filter with Cutoff frequency = {cutoff_freq}"
viz.view_filter_results(x, y, x, y_filtered, title=title)

In [10]:
cutoff_freq = 0.4

Filter = filters.FrequencyDomainFiltering()
Filter.filter(array=y, filter_technique='ideal', numExpansion=70, cutoff_freq=cutoff_freq, order=None)
y_filtered = Filter.getFiltered
title = f"Bessel filter with Cutoff frequency = {cutoff_freq}"
viz.view_filter_results(x, y, x, y_filtered, title=title)

In [11]:
cutoff_freq = 0.8

Filter = filters.FrequencyDomainFiltering()
Filter.filter(array=y, filter_technique='ideal', numExpansion=70, cutoff_freq=cutoff_freq, order=None)
y_filtered = Filter.getFiltered
title = f"Bessel filter with Cutoff frequency = {cutoff_freq}"
viz.view_filter_results(x, y, x, y_filtered, title=title)